<a href="https://colab.research.google.com/github/Mostafa318/DataAnalysisBootcamp/blob/Project1/AllPlayersFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import numpy as np 
import pandas as pd
!pip install aiohttp

In [ ]:
links = pd.read_parquet('allPlayersLink.parquet',engine='pyarrow')
players = pd.read_parquet('totalPlayers.parquet',engine='pyarrow')
links

,PlayersLink
0,/thibaut-courtois/profil/spieler/108390
1,/asmir-begovic/profil/spieler/33873
2,/marco-amelia/profil/spieler/16810
3,/jamal-blackman/profil/spieler/128898
4,/kurt-zouma/profil/spieler/157509
...,...
25893,/mamadou-sylla/profil/spieler/166683
25894,/john-guidetti/profil/spieler/98596
25895,/marc-tenas/profil/spieler/628472
25896,/giorgi-gagua/profil/spieler/550492


In [ ]:
players['playersId'].unique()[:10]
https://www.transfermarkt.com/asmir-begovic/leistungsdatendetails/spieler/33873/wettbewerb/GB1/saison/2022
https://www.transfermarkt.com/asmir-begovic/leistungsdatendetails/spieler/33873/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id=
https://www.transfermarkt.com/asmir-begovic/leistungsdatendetails/spieler/33873/saison//verein/0/liga/0/wettbewerb/GB1/pos/0/trainer_id/0/plus/1

array(['108390', '33873', '16810', '128898', '157509', '27511', '177476',
       '136610', '3160', '148657'], dtype=object)

In [ ]:
#main > main > div:nth-child(7) > div.large-8.columns > div > div.tm-tabs > a:nth-child(2)
# div class ="tm-tabs"
linksUnique =links['PlayersLink'].unique().tolist()
index=0
for index in range(len(linksUnique)):
    linksUnique[index] = 'https://www.transfermarkt.com'+linksUnique[index].split('profil')[0]+'leistungsdatendetails'+linksUnique[index].split('profil')[1]
# linksUnique[:10]

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
resp = requests.get('https://www.transfermarkt.com/thibaut-courtois/leistungsdatendetails/spieler/108390',headers=headers)
pageSoup = BeautifulSoup(resp.content,'html.parser')
link = pageSoup.find_all('div',{'class':'tm-tabs'})[0].find_all('a')[1].get('href')

In [ ]:
test = pd.read_parquet('continueLinks.parquet',engine='pyarrow')['links'].values.tolist()
test

['https://www.transfermarkt.com/saul-niguez/leistungsdatendetails/spieler/148928',
 'https://www.transfermarkt.com/charlie-webster/leistungsdatendetails/spieler/697278',
 'https://www.transfermarkt.com/harvey-vale/leistungsdatendetails/spieler/581675',
 'https://www.transfermarkt.com/jude-soonsup-bell/leistungsdatendetails/spieler/670857',
 'https://www.transfermarkt.com/raphael-varane/leistungsdatendetails/spieler/164770',
 'https://www.transfermarkt.com/bjorn-hardley/leistungsdatendetails/spieler/682562',
 'https://www.transfermarkt.com/alvaro-fernandez/leistungsdatendetails/spieler/811778',
 'https://www.transfermarkt.com/charlie-savage/leistungsdatendetails/spieler/686837',
 'https://www.transfermarkt.com/zidane-iqbal/leistungsdatendetails/spieler/686845',
 'https://www.transfermarkt.com/jadon-sancho/leistungsdatendetails/spieler/401173',
 'https://www.transfermarkt.com/alejandro-garnacho/leistungsdatendetails/spieler/811779',
 'https://www.transfermarkt.com/cristiano-ronaldo/leist

In [ ]:
uncatched1 = []

In [ ]:
import asyncio
import aiohttp
import time
import nest_asyncio
nest_asyncio.apply()

# test = pd.read_parquet('totalPlayers.parquet',engine='pyarrow')

playersLink=[]
index=0
# playersId , playersName , season_ , date_ , left_,joined_, mv_ , fee_ = [],[],[],[],[],[],[],[]

async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            # id1=url.split('profil/spieler/')[1]
            # name=test[test['playersId']==id1]['playersName'].iloc[0]
            pageSoup = BeautifulSoup(resp,'html.parser')
            link = pageSoup.find_all('div',{'class':'tm-tabs'})[0].find_all('a')[1].get('href')
            playersLink.append(link)
          
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))
        uncatched1.append(url)
connector = aiohttp.TCPConnector(limit=20)
    

async def main(urls):
    async with aiohttp.ClientSession(connector=connector) as session:
        ret = await asyncio.gather(*[get(url, session) for url in urls])
    # df = pd.DataFrame({'playersId':playersId,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ , 'mv':mv_ , 'fee':fee_})
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))


urls = uncatched
start = time.time()
asyncio.run(main(urls))
end = time.time()

print("Took {} seconds to pull {} websites.".format(end - start, len(urls)))

Finalized all. Return is a list of len 50 outputs.
Took 7.769670248031616 seconds to pull 50 websites.


In [ ]:
uncatched1=uncatched

In [ ]:
df = pd.DataFrame({'playersLink':playersLink})


In [ ]:
df.to_parquet('playersLinkuncatched.parquet',engine='pyarrow')

In [ ]:
pd.DataFrame({'links':linksUnique[2000:]}).to_parquet('continueLinks.parquet',engine='pyarrow')

In [ ]:
len(uncatched)

50

In [ ]:
p1 = pd.read_parquet('playersLink0-2000.parquet',engine='pyarrow')
p2 = pd.read_parquet('playersLink2000-4000.parquet',engine='pyarrow')
p3 = pd.read_parquet('playersLink4000-6000.parquet',engine='pyarrow')
p4 = pd.read_parquet('playersLink6000-8000.parquet',engine='pyarrow')
p5 = pd.read_parquet('playersLink8000-END.parquet',engine='pyarrow')
p6 = pd.read_parquet('playersLinkuncatched.parquet',engine='pyarrow')

In [ ]:
all = pd.concat([p1,p2,p3,p4,p5,p6],axis=0).reset_index(drop=True)

In [ ]:
all.to_parquet('allPlayersDetailsLink.parquet',engine='pyarrow')

In [2]:
all = pd.read_parquet('allPlayersDetailsLink.parquet',engine='pyarrow')
linkesUnique=all['playersLink'].tolist()
listlink=[]
for i in range(len(linkesUnique)):
  listlink.append('https://www.transfermarkt.com'+linkesUnique[i])

In [ ]:
listlink

In [122]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
url ='https://www.transfermarkt.com/thibaut-courtois/leistungsdatendetails/spieler/108390/plus/1?saison=&verein=&liga=&wettbewerb=ES1&pos=&trainer_id='
response = requests.get(url =url,headers=headers)
pageSoup = BeautifulSoup(response.content , 'html.parser')
part1  =pageSoup.find_all('div',{'class':'data-header__info-box '})


In [123]:
url

'https://www.transfermarkt.com/thibaut-courtois/leistungsdatendetails/spieler/108390/plus/1?saison=&verein=&liga=&wettbewerb=ES1&pos=&trainer_id='

In [125]:
playersId = url.split('/spieler/')[1].split('/')[0]
BirthDate = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[0].strip()
age=pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[1].strip(')')
birthPlace = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[1].span.text.strip()
citizenship = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[2].span.text.strip()
height  = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[0].span.text.strip()
position = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[1].span.text.strip()
# pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
currentValue = pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip()
timeValue = pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n')
size = len(pageSoup.find_all('table',{'class':'items'})[0].find_all('th'))
playersId

'108390'

In [132]:
playersId = url.split('/spieler/')[1].split('/')[0]
birthDate = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[0].strip()
age=pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[1].strip(')')
birthPlace = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[1].span.text.strip()
citizenship = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[2].span.text.strip()
height  = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[0].span.text.strip()
position = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[1].span.text.strip()
# pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
currentValue = pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip()
timeValue = pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n')



In [133]:
minutesPlayed

["2.520'",
 "3.240'",
 "3.420'",
 "3.060'",
 "2.430'",
 "3.330'",
 "3.330'",
 "3.262'"]

In [108]:
pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[0].text

IndexError: ignored

In [ ]:
season = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[0]
competition = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[1].img.get('title')
competitionUnicName = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1]
club =pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[3].img.get('title')
clubId = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0]
squad = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[4].text
appearances = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[5].text
ppg = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[6].text
goals = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[7].text
assists = 'goalKeeper'
ownGoal = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[8].text
subsituationsOn = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[9].text
subsituationsOff = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[10].text
yellowCards = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[11].text
secondYellowCards = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[12].text
redCards = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[13].text
penalyGoals = 'goalKeeper'
minutesPerGoal = 'goalKeeper'
cleanSheets = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[14].text
goalsConceded = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[15].text

minutesPlayed = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[0].find_all('td')[16].text

In [112]:
len(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr'))

8

In [141]:
#Final code
all = pd.read_parquet('allPlayersDetailsLink.parquet',engine='pyarrow')
linkesUnique=all['playersLink'].tolist()
listlink=[]
for i in range(len(linkesUnique)):
  listlink.append('https://www.transfermarkt.com'+linkesUnique[i])
playersId ,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age =[],[],[],[],[],[],[],[],[],[],[]
competitionUnicName,club,clubId,squad,appearances,ppg,goals,assists,ownGoal,subsituationsOn=[],[],[],[],[],[],[],[],[],[]
subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets=[],[],[],[],[],[],[],[],[]
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
columns = ['playersId','birthDate','birthPlace','citizenship','height','position','currentValue','timeValue','season','competition','age'\
           ,'competitionUnicName','club','clubId','squad','appearances','ppg','goals','assists','ownGoal','subsituationsOn'\
           ,'subsituationsOff','yellowCards','secondYellowCards','redCards','penalyGoals','minutesPerGoal','minutesPlayed','goalsConceded','cleanSheets']
data = [playersId ,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age,\
          competitionUnicName,club,clubId,squad,appearances,ppg,goals,assists,ownGoal,subsituationsOn\
          ,subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets]
for url in listlink[:10]:

  response = requests.get(url =url,headers=headers)
  pageSoup = BeautifulSoup(response.content , 'html.parser')
  print(url)
  size = len(pageSoup.find_all('table',{'class':'items'})[0].find_all('th'))
  for i in range(len(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr'))):
    if(size ==18): #not goal keeper
      playersId.append(url.split('/spieler/')[1].split('/')[0])
      try:
        birthDate.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[0].strip())
      except:
        birthDate.append(np.nan)
      try:
        age.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[1].strip(')'))
      except:
        age.append(np.nan)
      try:
        birthPlace.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[1].span.text.strip())
      except:
        birthPlace.append(np.nan)
      try:
        citizenship.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[2].span.text.strip())
      except: 
        citizenship.append(np.nan)
      try: 
        height.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[0].span.text.strip())
      except:
       height.append(np.nan)
      try:
        position.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[1].span.text.strip())
      except:
        position.append(np.nan)
  # pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
      try:
        currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
      except:
        currentValue.append(np.nan)
      try:
        timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
      except:
        timeValue.append(np.nan)
      season.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[0].text)
      competition.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[1].img.get('title'))
      competitionUnicName.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
      club.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[3].img.get('alt'))
      clubId.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
      squad.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[4].text)
      appearances.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[5].text)
      ppg.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[6].text)
      goals.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[7].text)
      assists.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[8].text)
      ownGoal.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[9].text)
      subsituationsOn.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[10].text)
      subsituationsOff.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[11].text)
      yellowCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[12].text)
      secondYellowCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[13].text)
      redCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[14].text)
      penalyGoals.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[15].text)
      minutesPerGoal.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[16].text)
      minutesPlayed.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[17].text)
      goalsConceded.append('not a goalKeeper')
      cleanSheets.append('not a goalKeeper')
    else:
      playersId.append(url.split('/spieler/')[1].split('/')[0])
      try:
        birthDate.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[0].strip())
      except:
        birthDate.append(np.nan)
      try:
        age.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].li.span.text.strip().split('(')[1].strip(')'))
      except:
        age.append(np.nan)
      try:
        birthPlace.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[1].span.text.strip())
      except:
        birthPlace.append(np.nan)
      try:
        citizenship.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0].find_all('li')[2].span.text.strip())
      except: 
        citizenship.append(np.nan)
      try: 
        height.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[0].span.text.strip())
      except:
       height.append(np.nan)
      try:
        position.append(pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[1].span.text.strip())
      except:
        position.append(np.nan)
  # pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
      try:
        currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
      except:
        currentValue.append(np.nan)
      try:
        timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
      except:
        timeValue.append(np.nan)
      season.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[0].text)
      competition.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[1].img.get('title'))
      competitionUnicName.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
      club.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[3].img.get('alt'))
      clubId.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
      squad.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[4].text)
      appearances.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[5].text)
      ppg.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[6].text)
      goals.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[7].text)
      assists.append('goalKeeper')
      ownGoal.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[8].text)
      subsituationsOn.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[9].text)
      subsituationsOff.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[10].text)
      yellowCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[11].text)
      secondYellowCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[12].text)
      redCards.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[13].text)
      penalyGoals.append('goalKeeper')
      minutesPerGoal.append('goalKeeper')
      cleanSheets.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[14].text)
      goalsConceded.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[15].text)

      minutesPlayed.append(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i].find_all('td')[16].text)
pd.DataFrame({columns[1]:data[1],columns[2]:data[2],columns[3]:data[3],columns[4]:data[4],columns[5]:data[5],columns[6]:data[6],columns[7]:data[7],\
              columns[8]:data[8],columns[9]:data[9],columns[10]:data[10],columns[11]:data[11],columns[12]:data[12],columns[13]:data[13],columns[14]:data[14],\
              columns[15]:data[15],columns[16]:data[16],columns[17]:data[17],columns[18]:data[18],columns[19]:data[19],columns[20]:data[20],columns[21]:data[21],\
              columns[22]:data[22],columns[23]:data[23],columns[24]:data[24],columns[25]:data[25],columns[26]:data[26],columns[27]:data[27],columns[28]:data[28],\
              columns[29]:data[29]})

https://www.transfermarkt.com/ola-aina/leistungsdatendetails/spieler/236490/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/nathan-ake/leistungsdatendetails/spieler/177476/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/jake-clarke-salter/leistungsdatendetails/spieler/316551/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/thibaut-courtois/leistungsdatendetails/spieler/108390/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/michael-hector/leistungsdatendetails/spieler/157490/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/kurt-zouma/leistungsdatendetails/spieler/157509/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/matt-miazga/leistungsdatendetails/spieler/245893/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
htt

ValueError: ignored

In [149]:
pd.DataFrame({columns[1]:data[1],columns[2]:data[2],columns[3]:data[3],columns[4]:data[4],columns[5]:data[5],columns[6]:data[6],columns[7]:data[7],\
              columns[8]:data[8],columns[9]:data[9],columns[10]:data[10],columns[11]:data[11],columns[12]:data[12],columns[13]:data[13],columns[14]:data[14],\
              columns[15]:data[15],columns[16]:data[16],columns[17]:data[17],columns[18]:data[18],columns[19]:data[19],columns[20]:data[20],columns[21]:data[21],\
              columns[22]:data[22],columns[23]:data[23],columns[24]:data[24],columns[25]:data[25],columns[26]:data[26],columns[27]:data[27],columns[28]:data[28],\
              columns[29]:data[29]})

,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age,...,subsituationsOn,subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets
0,"Oct 8, 1996",London,Nigeria,"1,84 m",Right-Back,€3.50m,"Nov 11, 2022",22/23,Serie A,26,...,8,5,5,-,-,-,834',834',not a goalKeeper,not a goalKeeper
1,"Oct 8, 1996",London,Nigeria,"1,84 m",Right-Back,€3.50m,"Nov 11, 2022",22/23,Italy Cup,26,...,1,-,-,-,-,-,-,118',not a goalKeeper,not a goalKeeper
2,"Oct 8, 1996",London,Nigeria,"1,84 m",Right-Back,€3.50m,"Nov 11, 2022",21/22,Serie A,26,...,4,10,4,-,-,-,-,1.443',not a goalKeeper,not a goalKeeper
3,"Oct 8, 1996",London,Nigeria,"1,84 m",Right-Back,€3.50m,"Nov 11, 2022",21/22,Italy Cup,26,...,-,1,-,-,-,-,-,201',not a goalKeeper,not a goalKeeper
4,"Oct 8, 1996",London,Nigeria,"1,84 m",Right-Back,€3.50m,"Nov 11, 2022",20/21,Premier League,26,...,-,7,3,-,-,-,1.335',2.670',not a goalKeeper,not a goalKeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,"Dec 19, 1997","Calgary, Alberta",England,"1,85 m",Centre-Back,€45.00m,"Mar 21, 2023",15/16,UEFA Youth League,25,...,1,-,3,-,-,-,757',757',not a goalKeeper,not a goalKeeper
401,"Dec 19, 1997","Calgary, Alberta",England,"1,85 m",Centre-Back,€45.00m,"Mar 21, 2023",14/15,Premier League 2,25,...,-,1,-,-,-,-,-,433',not a goalKeeper,not a goalKeeper
402,"Dec 19, 1997","Calgary, Alberta",England,"1,85 m",Centre-Back,€45.00m,"Mar 21, 2023",14/15,FA Youth Cup,25,...,-,1,1,-,-,-,-,549',not a goalKeeper,not a goalKeeper
403,"Dec 19, 1997","Calgary, Alberta",England,"1,85 m",Centre-Back,€45.00m,"Mar 21, 2023",14/15,UEFA Youth League,25,...,1,4,-,-,-,-,-,522',not a goalKeeper,not a goalKeeper


In [200]:
all = pd.read_parquet('allPlayersDetailsLink.parquet',engine='pyarrow')
linkesUnique=all['playersLink'].tolist()
listlink=[]
for i in range(len(linkesUnique)):
  listlink.append('https://www.transfermarkt.com'+linkesUnique[i])
uncatched = []

In [202]:
import asyncio
import aiohttp
import time
import nest_asyncio
nest_asyncio.apply()
# uncatched=[]
playersId ,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age =[],[],[],[],[],[],[],[],[],[],[]
competitionUnicName,club,clubId,squad,appearances,ppg,goals,assists,ownGoal,subsituationsOn=[],[],[],[],[],[],[],[],[],[]
subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets=[],[],[],[],[],[],[],[],[]
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
columns = ['playersId','birthDate','birthPlace','citizenship','height','position','currentValue','timeValue','season','competition','age'\
           ,'competitionUnicName','club','clubId','squad','appearances','ppg','goals','assists','ownGoal','subsituationsOn'\
           ,'subsituationsOff','yellowCards','secondYellowCards','redCards','penalyGoals','minutesPerGoal','minutesPlayed','goalsConceded','cleanSheets']
data = [playersId ,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age,\
          competitionUnicName,club,clubId,squad,appearances,ppg,goals,assists,ownGoal,subsituationsOn\
          ,subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets]
async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            pageSoup = BeautifulSoup(resp , 'html.parser')
            head = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0]
            head2 = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1]
            
            # print(url)
            size = len(pageSoup.find_all('table',{'class':'items'})[0].find_all('th'))
            for i in range(len(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr'))):
              head3 = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i]
              if(size ==18): #not goal keeper
                try:
                  playersId.append(url.split('/spieler/')[1].split('/')[0])
                except:
                  playersId.append(np.nan)
                try:
                  birthDate.append(head.li.span.text.strip().split('(')[0].strip())
                except:
                  birthDate.append(np.nan)
                try:
                  age.append(head.li.span.text.strip().split('(')[1].strip(')'))
                except:
                  age.append(np.nan)
                try:
                  birthPlace.append(head.find_all('li')[1].span.text.strip())
                except:
                  birthPlace.append(np.nan)
                try:
                  citizenship.append(head.find_all('li')[2].span.text.strip())
                except: 
                  citizenship.append(np.nan)
                try: 
                  height.append(head2.find_all('li')[0].span.text.strip())
                except:
                  height.append(np.nan)
                try:
                  position.append(head2.find_all('li')[1].span.text.strip())
                except:
                  position.append(np.nan)
            # pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
                try:
                  currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
                except:
                  currentValue.append(np.nan)
                try:
                  timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
                except:
                  timeValue.append(np.nan)
                try:
                  season.append(head3.find_all('td')[0].text)
                except:
                  season.append(np.nan)
                try:
                  competition.append(head3.find_all('td')[1].img.get('title'))
                except:
                  competition.append(np.nan)
                try:
                  competitionUnicName.append(head3.find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
                except:
                  competition.append(np.nan)

                
                try:
                  club.append(head3.find_all('td')[3].img.get('alt'))
                except:
                  club.append(np.nan)
                try:
                  clubId.append(head3.find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
                except:
                  clubId.append(np.nan)
                try:

                  squad.append(head3.find_all('td')[4].text)
                except:
                  squad.append(np.nan)
                try:  
                  appearances.append(head3.find_all('td')[5].text)
                except:
                  appearances.append(np.nan)
                try:
            
                   ppg.append(head3.find_all('td')[6].text)
                except:
                  ppg.append(np.nan)
                try:

                  goals.append(head3.find_all('td')[7].text)
                except:
                  goals.append(np.nan)
                try:
                  assists.append(head3.find_all('td')[8].text)
                except:
                  assists.append(np.nan)
                try:
                  ownGoal.append(head3.find_all('td')[9].text)
                except:
                  ownGoal.apppend(np.nan)
                try:  
                  subsituationsOn.append(head3.find_all('td')[10].text)
                except:
                  subsituationsOn.append(np.nan)
                try:  
                  subsituationsOff.append(head3.find_all('td')[11].text)
                except:
                  subsituationsOff.append(np.nan)
                try:  
                  yellowCards.append(head3.find_all('td')[12].text)
                except:
                  yellowCards.append(np.nan)
                try:  
                  secondYellowCards.append(head3.find_all('td')[13].text)
                except:
                  secondYellowCards.append(np.nan)
                try:  
                  redCards.append(head3.find_all('td')[14].text)
                except:
                  redCards.append(np.nan)  
                try:  
                  penalyGoals.append(head3.find_all('td')[15].text)
                except:
                  penalyGoals.append(np.nan)
                try:  
                  minutesPerGoal.append(head3.find_all('td')[16].text)
                except:
                  minutesPerGoal.append(np.nan)
                try:  
                    minutesPlayed.append(head3.find_all('td')[17].text)
                except:
                  minutesPlayed.append(np.nan)
                
                goalsConceded.append('not a goalKeeper')
                cleanSheets.append('not a goalKeeper')
              else:
                try:
                  birthDate.append(head.li.span.text.strip().split('(')[0].strip())
                except:
                  birthDate.append(np.nan)
                try:
                  age.append(head.li.span.text.strip().split('(')[1].strip(')'))
                except:
                  age.append(np.nan)
                try:
                  birthPlace.append(head.find_all('li')[1].span.text.strip())
                except:
                  birthPlace.append(np.nan)
                try:
                  citizenship.append(head.find_all('li')[2].span.text.strip())
                except: 
                  citizenship.append(np.nan)
                try: 
                  height.append(head2.find_all('li')[0].span.text.strip())
                except:
                  height.append(np.nan)
                try:
                  position.append(head2.find_all('li')[1].span.text.strip())
                except:
                  position.append(np.nan)
            # pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
                try:
                  currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
                except:
                  currentValue.append(np.nan)
                try:
                  timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
                except:
                  timeValue.append(np.nan)
                try:
                  season.append(head3.find_all('td')[0].text)
                except:
                  season.append(np.nan)
                try:
                  competition.append(head3.find_all('td')[1].img.get('title'))
                except:
                  competition.append(np.nan)
                try:
                  competitionUnicName.append(head3.find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
                except:
                  competition.append(np.nan)

                
                try:
                  club.append(head3.find_all('td')[3].img.get('alt'))
                except:
                  club.append(np.nan)
                try:
                  clubId.append(head3.find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
                except:
                  clubId.append(np.nan)
                try:

                  squad.append(head3.find_all('td')[4].text)
                except:
                  squad.append(np.nan)
                try:  
                  appearances.append(head3.find_all('td')[5].text)
                except:
                  appearances.append(np.nan)
                try:
            
                   ppg.append(head3.find_all('td')[6].text)
                except:
                  ppg.append(np.nan)
                try:

                  goals.append(head3.find_all('td')[7].text)
                except:
                  goals.append(np.nan)
                assists.append('goalKeeper')
                try:
                  ownGoal.append(head3.find_all('td')[8].text)
                except:
                  ownGoal.append(np.nan)
                try:
                  subsituationsOn.append(head3.find_all('td')[9].text)
                except:
                  subsituationsOn.append(np.nan)
                try:
                  subsituationsOff.append(head3.find_all('td')[10].text)
                except:
                  subsituationsOff.append(np.nan)
                try:
                  yellowCards.append(head3.find_all('td')[11].text)
                except:
                  yellowCards.append(np.nan)
                try:
                  secondYellowCards.append(head3.find_all('td')[12].text)
                except:
                  secondYellowCards.append(np.nan)

                try:
                  redCards.append(head3.find_all('td')[13].text)
                except:
                  redCards.append(head3.find_all('td')[13].text)
                penalyGoals.append('goalKeeper')
                try:
                  cleanSheets.append(head3.find_all('td')[14].text)
                except:
                  cleanSheets.append(np.nan)

                minutesPerGoal.append('goalKeeper')
                try:
                  goalsConceded.append(head3.find_all('td')[15].text)
                except:
                  goalsConceded.append(np.nan)
                try:
                  minutesPlayed.append(head3.find_all('td')[16].text)
                except:
                  minutesPlayed.append(np.nan)    
          
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))
        uncatched.append(url)
connector = aiohttp.TCPConnector(limit=50)
    

async def main(urls):
    async with aiohttp.ClientSession(connector=connector) as session:
        ret = await asyncio.gather(*[get(url, session) for url in urls])
    # df = pd.DataFrame({'playersId':playersId,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ , 'mv':mv_ , 'fee':fee_})
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))


urls = listlink[:1000]
start = time.time()
asyncio.run(main(urls))
end = time.time()

print("Took {} seconds to pull {} websites.".format(end - start, len(urls)))

Unable to get url https://www.transfermarkt.com/david-silva/leistungsdatendetails/spieler/35518/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1 due to <class 'aiohttp.client_exceptions.ServerDisconnectedError'>.
Unable to get url https://www.transfermarkt.com/brandon-barker/leistungsdatendetails/spieler/214927/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1 due to <class 'aiohttp.client_exceptions.ServerDisconnectedError'>.
Unable to get url https://www.transfermarkt.com/patrick-roberts/leistungsdatendetails/spieler/225452/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1 due to <class 'aiohttp.client_exceptions.ServerDisconnectedError'>.
Unable to get url https://www.transfermarkt.com/raheem-sterling/leistungsdatendetails/spieler/134425/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1 due to <class 'aiohttp.client_exceptions.ServerDisconnectedError'>.
Unable to get url https://www.transfermarkt.com/aleix-garcia/leistungsdatendetails

In [194]:
df= pd.DataFrame({columns[1]:data[1],columns[2]:data[2],columns[3]:data[3],columns[4]:data[4],columns[5]:data[5],columns[6]:data[6],columns[7]:data[7],\
              columns[8]:data[8],columns[9]:data[9],columns[10]:data[10],columns[11]:data[11],columns[12]:data[12],columns[13]:data[13],columns[14]:data[14],\
              columns[15]:data[15],columns[16]:data[16],columns[17]:data[17],columns[18]:data[18],columns[19]:data[19],columns[20]:data[20],columns[21]:data[21],\
              columns[22]:data[22],columns[23]:data[23],columns[24]:data[24],columns[25]:data[25],columns[26]:data[26],columns[27]:data[27],columns[28]:data[28],\
              columns[29]:data[29]})
df

,birthDate,birthPlace,citizenship,height,position,currentValue,timeValue,season,competition,age,...,subsituationsOn,subsituationsOff,yellowCards,secondYellowCards,redCards,penalyGoals,minutesPerGoal,minutesPlayed,goalsConceded,cleanSheets
0,"Nov 30, 1995",Hazel Grove,England,"1,78 m",midfield,NaN,NaN,21/22,National League North,27,...,2,4,1,-,-,-,1.055',1.055',not a goalKeeper,not a goalKeeper
1,"Nov 30, 1995",Hazel Grove,England,"1,78 m",midfield,NaN,NaN,20/21,National League,27,...,4,2,5,-,-,-,-,1.096',not a goalKeeper,not a goalKeeper
2,"Nov 30, 1995",Hazel Grove,England,"1,78 m",midfield,NaN,NaN,19/20,National League,27,...,-,1,-,-,-,-,-,266',not a goalKeeper,not a goalKeeper
3,"Nov 30, 1995",Hazel Grove,England,"1,78 m",midfield,NaN,NaN,19/20,League Two,27,...,-,-,-,-,-,-,-,-,not a goalKeeper,not a goalKeeper
4,"Nov 30, 1995",Hazel Grove,England,"1,78 m",midfield,NaN,NaN,19/20,EFL Cup,27,...,-,-,-,-,-,-,-,-,not a goalKeeper,not a goalKeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19513,"Oct 21, 1986",Prizren,Switzerland,"1,81 m",Central Midfield,NaN,NaN,05/06,Swiss Cup,36,...,-,-,-,-,-,-,90',90',not a goalKeeper,not a goalKeeper
19514,"Oct 21, 1986",Prizren,Switzerland,"1,81 m",Central Midfield,NaN,NaN,05/06,UEFA-Cup Qualifikation,36,...,1,-,-,-,-,-,-,7',not a goalKeeper,not a goalKeeper
19515,"Oct 21, 1986",Prizren,Switzerland,"1,81 m",Central Midfield,NaN,NaN,04/05,Super League,36,...,8,1,-,-,-,-,-,99',not a goalKeeper,not a goalKeeper
19516,"Oct 21, 1986",Prizren,Switzerland,"1,81 m",Central Midfield,NaN,NaN,04/05,Swiss Cup,36,...,2,-,-,-,-,-,-,6',not a goalKeeper,not a goalKeeper


In [204]:
len(uncatched)

606

In [ ]:
for i in range(30):
  if not  (len(data[2])==len(data[i])):
    print(i)

In [169]:
res =requests.get(listlink[1],headers=headers)
pageSoup = BeautifulSoup(res.content , 'html.parser')
head = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[0]
head2 = pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1]
head3 = pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr')[i]
# print(url)
size = len(pageSoup.find_all('table',{'class':'items'})[0].find_all('th'))
for i in range(len(pageSoup.find_all('table',{'class':'items'})[0].tbody.find_all('tr'))):
  
  if(size ==18): #not goal keeper
    playersId.append(url.split('/spieler/')[1].split('/')[0])
    
    try:
      birthDate.append(head.li.span.text.strip().split('(')[0].strip())
    except:
      birthDate.append(np.nan)
    try:
      age.append(head.li.span.text.strip().split('(')[1].strip(')'))
    except:
      age.append(np.nan)
    try:
      birthPlace.append(head.find_all('li')[1].span.text.strip())
    except:
      birthPlace.append(np.nan)
    try:
      citizenship.append(head.find_all('li')[2].span.text.strip())
    except: 
      citizenship.append(np.nan)
    try: 
      height.append(head2.find_all('li')[0].span.text.strip())
    except:
      height.append(np.nan)
    try:
      position.append(head2.find_all('li')[1].span.text.strip())
    except:
      position.append(np.nan)
# pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
    try:
      currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
    except:
      currentValue.append(np.nan)
    try:
      timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
    except:
      timeValue.append(np.nan)
    season.append(head3.find_all('td')[0].text)
    competition.append(head3.find_all('td')[1].img.get('title'))
    competitionUnicName.append(head3.find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
    club.append(head3.find_all('td')[3].img.get('alt'))
    clubId.append(head3.find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
    squad.append(head3.find_all('td')[4].text)
    appearances.append(head3.find_all('td')[5].text)
    ppg.append(head3.find_all('td')[6].text)
    goals.append(head3.find_all('td')[7].text)
    assists.append(head3.find_all('td')[8].text)
    ownGoal.append(head3.find_all('td')[9].text)
    subsituationsOn.append(head3.find_all('td')[10].text)
    subsituationsOff.append(head3.find_all('td')[11].text)
    yellowCards.append(head3.find_all('td')[12].text)
    secondYellowCards.append(head3.find_all('td')[13].text)
    redCards.append(head3.find_all('td')[14].text)
    penalyGoals.append(head3.find_all('td')[15].text)
    minutesPerGoal.append(head3.find_all('td')[16].text)
    minutesPlayed.append(head3.find_all('td')[17].text)
    goalsConceded.append('not a goalKeeper')
    cleanSheets.append('not a goalKeeper')
  else:
    try:
      birthDate.append(head.li.span.text.strip().split('(')[0].strip())
    except:
      birthDate.append(np.nan)
    try:
      age.append(head.li.span.text.strip().split('(')[1].strip(')'))
    except:
      age.append(np.nan)
    try:
      birthPlace.append(head.find_all('li')[1].span.text.strip())
    except:
      birthPlace.append(np.nan)
    try:
      citizenship.append(head.find_all('li')[2].span.text.strip())
    except: 
      citizenship.append(np.nan)
    try: 
      height.append(head2.find_all('li')[0].span.text.strip())
    except:
      height.append(np.nan)
    try:
      position.append(head2.find_all('li')[1].span.text.strip())
    except:
      position.append(np.nan)
# pageSoup.find_all('div',{'class':'data-header__details'})[0].find_all('ul')[1].find_all('li')[2].span.text.strip()
    try:
      currentValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[0].strip())
    except:
      currentValue.append(np.nan)
    try:
      timeValue.append(pageSoup.find_all('div',{'class':'data-header__box--small'})[0].text.split('Last update')[1].strip(':\n'))
    except:
      timeValue.append(np.nan)
    season.append(head3.find_all('td')[0].text)
    competition.append(head3.find_all('td')[1].img.get('title'))
    competitionUnicName.append(head3.find_all('td')[2].a.get('href').split('/saison_id')[0].split('/')[-1])
    club.append(head3.find_all('td')[3].img.get('alt'))
    clubId.append(head3.find_all('td')[3].a.get('href').split('verein/')[1].split('/')[0])
    squad.append(head3.find_all('td')[4].text)
    appearances.append(head3.find_all('td')[5].text)
    ppg.append(head3.find_all('td')[6].text)
    goals.append(head3.find_all('td')[7].text)
    assists.append('goalKeeper')
    ownGoal.append(head3.find_all('td')[8].text)
    subsituationsOn.append(head3.find_all('td')[9].text)
    subsituationsOff.append(head3.find_all('td')[10].text)
    yellowCards.append(head3.find_all('td')[11].text)
    secondYellowCards.append(head3.find_all('td')[12].text)
    redCards.append(head3.find_all('td')[13].text)
    penalyGoals.append('goalKeeper')
    minutesPerGoal.append('goalKeeper')
    cleanSheets.append(head3.find_all('td')[14].text)
    goalsConceded.append(head3.find_all('td')[15].text)

    minutesPlayed.append(head3.find_all('td')[16].text)

IndexError: ignored

In [192]:
pd.read_parquet('AllplayersTransfer.parquet',engine='pyarrow')

,playersId,season,date,left,joined,mv,fee
0,303254,21/22,"Jul 1, 2021","[Chelsea, 631]","[AC Milan, 631]",€28.00m,€31.60m
1,303254,20/21,"Jun 30, 2021","[AC Milan, 5]","[Chelsea, 5]",€28.00m,End of loan
2,303254,20/21,"Jan 22, 2021","[Chelsea, 631]","[AC Milan, 631]",€17.00m,Loan fee:€600k
3,303254,18/19,"May 31, 2019","[Derby, 22]","[Chelsea, 22]",€7.00m,End of loan
4,303254,18/19,"Aug 6, 2018","[Chelsea, 631]","[Derby, 631]",€1.00m,loan transfer
...,...,...,...,...,...,...,...
88246,668277,22/23,"Jun 30, 2023","[Sanluqueño, 21322]","[Depor. Alavés B, 21322]",€100k,End of loan
88247,668277,22/23,"Jan 31, 2023","[Depor. Alavés B, 8880]","[Sanluqueño, 8880]",€100k,loan transfer
88248,668277,20/21,"Jul 1, 2020","[Alavés U19, 44202]","[Depor. Alavés B, 44202]",€100k,-
88249,668277,19/20,"Jul 1, 2019","[Deportivo Alavés, 8881]","[Alavés U19, 8881]",-,-
